<a href="https://colab.research.google.com/github/ArpitJain25/AI-RAG/blob/main/RAG_LangChain_Gemini_by_APJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain
!pip install -U langchain-community
!pip install -U langchain_openai
#!pip install faiss-cpu
!pip install pypdf
!pip install Chromadb
%pip install --upgrade --quiet  langchain-google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import google.generativeai as genai
#from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from google.colab import userdata
import os
GOOGLE_API_KEY=userdata.get('apj')
genai.configure(api_key=GOOGLE_API_KEY)
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
pdfloader = PyPDFLoader("/content/Modi-Ki-Guarantee-Sankalp-Patra-English.pdf")
pdfpages = pdfloader.load_and_split()
print(pdfpages[1].page_content)

2


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pdfpages)
texts = text_splitter.split_text(context)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_keyj=GOOGLE_API_KEY)

In [ ]:
vector_store = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})


In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,
                             temperature=0.5)

In [ ]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=vector_store,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [ ]:
question = "What are the most important achievements of Modi Government in last 10 years?"
result = qa_chain.invoke({"query": question})
Markdown(result["result"])

Some of the most important achievements of the Modi Government in the last 10 years include:
- Construction of a new National War Memorial and new National Police Memorial to honour our self-sacrificing soldiers and police personnel
- Zero major terror attacks in any city since 2014
- Revoked Article 370 leading to a significant reduction in violence in Jammu and Kashmir
- 52% reduction in Left-Wing Extremism-related violence
- 71% reduction in insurgency related violence in the Northeast resulted in the removal of the AFSPA from most areas
- We have enacted the Bhartiya Nyay Sanhita which focuses on justice instead of punishments
- Increased the non-fossil fuel energy capacity by 2.5 times to 180 GW
- Achieved 100% electrification by providing electricity connection to 2.8+ crore families under Saubhagya
- Unprecedented improvements in connectivity of the Northeast by expansion of roads, bridges, railways and airports
- Achieved the fastest 5G rollout in the world with the most affordable data prices
- Cleaned up public procurement through GEM
- Evacuation and safe return of 1.5+ crore citizens in distress situations
- Successfully hosted the historic G20 Summit, achieving 100% consensus on the New Delhi Declaration
- Bharat becomes Voice of the Global South
- Digital India Stack, including UPI, becomes global
- Trading started in INR with 27 partner countries
- Leading the development of the India-Middle East-Europe Economic Corridor
- Provided COVID assistance, including 30+ crore COVID-19 vaccines and medicines, to 100 countries through Vaccine Maitri
- Ensuring freedom of navigation and maritime security in the Indian Ocean  
- Became the first country in the world to land successfully near the South Pole of the moon through Chandrayaan 3
- The UN declared 21st June as the International Day of Yoga at Bharat’s initiative
- The UN International Year of Millets at Bharat’s initiative brought Shree Anna into the global discourse
Thanks for asking!